In [ ]:
# To use only Google Colab
# ! pip install matplotlib --upgrade

# Unidad II. Regresiones y reducción de dimensionalidad.

## Modelos de regresión lineales

Los modelos de regresión lineales:
- predicen una o más variable *dependientes*
- como una combinación lineal de
  - un conjunto de variables *independientes*
- $Y = \alpha + \beta_{1}X_{1} + \beta_{2}X_{2} + ... + \beta_{k}X_{k} + E$
  - $Y$ es la variable *dependiente*.
  - $X_1, X_2, ...$ son las variables *independientes*.
  - $\beta_1, \beta_2, ...$ son los *coeficientes*.
  - $\alpha$ es la ordenada al origen.
  - $E$ es un término que representa el error.
- El problema del modelo:
  - Es hallar $\alpha, \beta_1, \beta_2$
  - Esto suele hacerse mediante el criterio de cuadrados mínimos:
  - minimizar $ \sum_{i=1}^{N}(Y_{i}-\hat{Y}_{i})^{2} = \sum_{i=1}^{N} E_{i}^{2}$
- El modelo asume:
  - *Linealidad*: La variable dependiente se obtiene por una combinación lineal
    de las otras.
  - Independencia de los residuos
  - Homocedasticidad: Igualdad de varianza para cada variable independiente.
  - Normalidad: Los residuos se distribuyen de manera normal con $\mu = 0$
  - No-colinealidad: Las variables independientes no pueden ser combinaciones
    lineales unas de otras, no debe haber correlación entre ellas.

Vamos a simular un modelo lineal completo.

In [ ]:
import numpy as np
import scipy.stats as st

# Simular variables aleatorias
nvars = 2
nvalues = 1000
variables = np.random.rand(nvars, nvalues)

# Simular la intersección
alpha = np.random.rand()

# Simulación de los coeficientes
betas = np.random.rand(nvars)

# simular los errores
errors = 0.05 * st.norm.rvs(size=nvalues)

In [ ]:
model = alpha + np.matmul(betas, variables)  + errors
model[:4]


In [ ]:
import statsmodels.api as sm

exog = sm.add_constant(variables.T)

# Ordinal Least Square
reg_mod = sm.OLS(
    endog = model,
    exog = exog
)

fitted = reg_mod.fit()


In [ ]:
# alpha, beta1, beta2
print(f"Fitted coefficients: {fitted.params}")
print(f"Real coefficients: {[alpha, betas]}")


In [ ]:
fitted.summary()

In [ ]:
var_with_intercept = sm.add_constant(variables.T)
print("Variables with estimated alpha:")
print(var_with_intercept[:4,:])

print(f"predicted:\n{fitted.predict(var_with_intercept[:4,:])}")
print(f"Real values:\n{model[:4]}")


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(
    model,
    fitted.predict(var_with_intercept)
)
plt.xlabel("Real data")
plt.ylabel("Predicted")
plt.title(f"Coeficiente de determinación: $R^2 = {fitted.rsquared:0.3f}$")

In [ ]:
_ = plt.hist(fitted.resid, bins = 50)

### Coeficientes de determinación

Es el valor $R^2$ de la regresión. 
- Es la proporción de la varianza de la variables de respuesta que puede ser
  predicha a partir de las variables explicativas.
- Varía entre 0 y 1.
  - Cuanto mayor es, menos incerteza se tiene de los valores reales a partir del
   modelo.
- Suma de los cuadrado de los datos:
  - $SS_{data} = \sum_i{(y_i-\bar{y})^2}$
- Suma de los cuadrados de las predicciones:
  - $SS_{pred} = \sum_i{(fit(y_i)-\bar{y})^2}$
- $R^2 = \frac{SS_{pred}}{SS_{data}}$

Veamos como cambia el $R^2$ con el error del modelo.

In [ ]:
nvars <- 2
nvalues <- 50
variables <- np.random.rand(nvars, nvalues)
exog = sm.add_constant(variables.T)
alpha <- np.random.rand()
betas <- np.random.rand(nvars)
fig, axes = plt.subplots(ncols=2, nrows=2)
axes = axes.flatten()
for i, err in enumerate([0.05, 0.2, 0.5, 1]):
  errors = st.norm.rvs(size=nvalues) * err
  y = alpha + betas@variables + errors
  reg_mod = sm.OLS(endog = y, exog = exog)
  fitted = reg_mod.fit()
  fval = fitted.fittedvalues
  axes[i].scatter(y,fval)
  axes[i].set_title(f"$R^2$: {fitted.rsquared:0.4f}")
fig.tight_layout()

In [ ]:
fitted.resid

### Mínimos Cuadrados Ponderados.

En el caso de no cumplirse con la homocedacia
- es posible darle a cada observación un peso inversamente proporcional a la
  varianza para ese variable.
- Regresion por minimos cuadrados ponderados.
- Lo óptimo es conocer la varianza de los residuos
  - Muy dificil de conocer
- Se puede estimar a partir de los residuos.


## Ridge Regression

- Esta regresión es mucho más robusta a la presencia de variables
  explicativas correlacionadas (colineales).



### Buscando colinearidad

Correlación:
- Medidas de variación conjunta entre dos variables aleatorias.
- Es una medida sin unidades que varía entre -1 y 1.
    - más fácil de trabajar e interpretar.


In [ ]:
import sklearn.datasets as datasets

iris = datasets.load_iris(as_frame=True)

iris_m = iris["frame"].iloc[:, :4].copy()
iris_m

# Queremos estimar sepal_length a patir de las otras variables.
iris_m.corr()

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold

# Defino los variables dependientes e independientes.
x = iris_m[["sepal width (cm)", "petal length (cm)", "petal width (cm)"]]
y = iris_m["sepal length (cm)"]

# Defino el modelo
model = Ridge()

# Hago el ajuste del modelo
fitted = model.fit(x, y)

plt.scatter(y, fitted.predict(x))